<a href="https://colab.research.google.com/github/meganificent/Language-Style-Transfer/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.	Insert English phrase
a.	Tokenize English phrase --> get POS array of tuple
2.	Translate English phrase to Spanish using Google Translate
3.	Token Spanish phrase
4.	Spaghetti tokenized Spanish phrase --> output array of tuples with Spanish word as the key and pos as the value
5.	Go through each key of POS English phrase and compare to keys of POS Spanish phrases --> move all English object intervals so that they are in the same positions as their translated Spanish word counterparts
6.	Merge rearranged array of objects so that their keys produce one potentially grammatically incoherent English sentence

Special cases: "a" in spanish, "to" for location in English --> brainstorm placeholders for these kinds of things

In [177]:
import matplotlib
matplotlib.use('Agg')
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import numpy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [178]:
from __future__ import print_function

In [179]:
import spaghetti as sp
import nltk
nltk.download('cess_esp')
mytagger = sp.CESSTagger()
mytagger_uni = mytagger.uni
mytagger_bi = mytagger.bi

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


# New Section

In [196]:
from nltk.corpus import wordnet
syns = wordnet.synsets("up")
print(syns[2].lemmas()[0].antonyms())

[]


In [183]:
	synonyms = []
	antonyms = []

	for syn in wordnet.synsets("none"):
		for l in syn.lemmas():
			synonyms.append(l.name())
			if l.antonyms():
				 antonyms.append(l.antonyms()[0].name())

	print(set(synonyms))
	print(set(antonyms))

{'none'}
set()


### **Google Translate**

In [ ]:
!pip install googletrans==3.1.0a0
import googletrans
print(googletrans.LANGUAGES)
from googletrans import Translator

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [ ]:
translator = Translator(service_urls=['translate.googleapis.com'])


In [ ]:
result = translator.translate("el perro blanco corrió", dest="en")
print(result.text)
print(result.src)
print(result.dest)


the white dog ran
es
en


In [300]:
spanglishMachine("white coffee cup spilled it")

Original: [('white', 'JJ'), ('coffee', 'NN'), ('cup', 'NN'), ('spilled', 'VBD'), ('it', 'PRP')]
Remove Pronouns Before Verbs: [('white', 'JJ'), ('coffee', 'NN'), ('cup', 'NN'), ('spilled', 'VBD'), ('it', 'PRP')]
Pronoun+Verb+Noun: [('it', 'PRP'), ('spilled', 'VBD'), ('white', 'JJ'), ('coffee', 'NN'), ('cup', 'NN')]
Adjectives after Nouns: [('it', 'PRP'), ('spilled', 'VBD'), ('coffee', 'NN'), ('cup', 'NN'), ('white', 'JJ')]
Add 'the' [('it', 'PRP'), ('spilled', 'VBD'), ('the', 'DT'), ('coffee', 'NN'), ('cup', 'NN'), ('white', 'JJ')]
Attributive Nouns: [('it', 'PRP'), ('spilled', 'VBD'), ('the', 'DT'), ('cup', 'NN'), ('of', 'IN'), ('coffee', 'NN'), ('white', 'JJ')]


In [301]:
spanglishMachine("Megan wrote it")

Original: [('Megan', 'NNP'), ('wrote', 'VBD'), ('it', 'PRP')]
Remove Pronouns Before Verbs: [('Megan', 'NNP'), ('wrote', 'VBD'), ('it', 'PRP')]
Pronoun+Verb+Noun: [('it', 'PRP'), ('wrote', 'VBD'), ('Megan', 'NNP')]
Adjectives after Nouns: [('it', 'PRP'), ('wrote', 'VBD'), ('Megan', 'NNP')]
Add 'the' [('it', 'PRP'), ('wrote', 'VBD'), ('Megan', 'NNP')]
Attributive Nouns: [('it', 'PRP'), ('wrote', 'VBD'), ('Megan', 'NNP')]


In [302]:
spanglishMachine("I write but the hot sun stays down")

Original: [('I', 'PRP'), ('write', 'VBP'), ('but', 'CC'), ('the', 'DT'), ('hot', 'JJ'), ('sun', 'NN'), ('stays', 'VBZ'), ('down', 'RP')]
Remove Pronouns Before Verbs: [('write', 'VBP'), ('but', 'CC'), ('the', 'DT'), ('hot', 'JJ'), ('sun', 'NN'), ('stays', 'VBZ'), ('down', 'RP')]
Pronoun+Verb+Noun: [('write', 'VBP'), ('but', 'CC'), ('the', 'DT'), ('hot', 'JJ'), ('sun', 'NN'), ('stays', 'VBZ'), ('down', 'RP')]
Adjectives after Nouns: [('write', 'VBP'), ('but', 'CC'), ('the', 'DT'), ('sun', 'NN'), ('hot', 'JJ'), ('stays', 'VBZ'), ('down', 'RP')]
Add 'the' [('write', 'VBP'), ('but', 'CC'), ('the', 'DT'), ('sun', 'NN'), ('hot', 'JJ'), ('stays', 'VBZ'), ('down', 'RP')]
Attributive Nouns: [('write', 'VBP'), ('but', 'CC'), ('the', 'DT'), ('sun', 'NN'), ('hot', 'JJ'), ('stays', 'VBZ'), ('down', 'RP')]


In [286]:
def spanglishMachine(string): 
  origTokenized = nltk.word_tokenize(string)

  #determine language of inserted string and feed into correct translation function
  
  translated = translator.translate(string, dest = "en") #initially translate to english
  if translated.src == "en": #if the original language was english
    tagged = nltk.pos_tag(origTokenized) #tag POS using nltk
    print("Original:", tagged)
    toSpanishStyle(tagged) #transfer to Spanish

  else: #if original languagse was spanish
    tagged = sp.pos_tag(origTokenized) #tag POS using spaghetti
    print("Original:", tagged)
    toEnglishStyle(tagged) #transfer to English



In [240]:
fi = ["hi", "my", "tired", "mind"]
fi[3:3] = "beautiful", "lost"
print(fi)
fi[4:4] = fi[1:3]
print(fi)
del fi[1:3]
print(fi)
print(len(fi))

['hi', 'my', 'tired', 'beautiful', 'lost', 'mind']
['hi', 'my', 'tired', 'beautiful', 'my', 'tired', 'lost', 'mind']
['hi', 'beautiful', 'my', 'tired', 'lost', 'mind']
6


In [299]:
def toSpanishStyle(taggedWords):
  #Rule 1: Remove P
  #taggedWords = [word for word in taggedWords if word[0] != "I"]

  #Rule 1: Remove Pronouns Directly Before Conjugated Verbs
  idx = 0
  while idx < len(taggedWords) - 1:
    if taggedWords[idx][1] == "PRP":
      if taggedWords[idx+1][1][:2] == "VB":
        del taggedWords[idx]
    idx += 1
  print("Remove Pronouns Before Verbs:", taggedWords)

  #Rule 2: Double Negatives
  #for word in taggedWords:
    #if englishNegAndAff[0].index(word[0]):
      #print("aah this scares me")


  #Rule 3: Pronoun + Verb + Noun
  idx = 0
  nounStart = 0
  nounEnd = 0
  while idx < len(taggedWords) - 1:
    if taggedWords[idx][1][:2] == "VB": #if current word is a verb
      #print(idx)
      if taggedWords[idx+1][1] == "PRP": #if there's a pronoun directly after the verb
        if idx-1 >= 0: #to avoid runtime errors
          if taggedWords[idx-1][1][:2] == "NN": #if there's a noun right before the verb
            nounStart = idx-1 #start of noun and its modifiers
            nounEnd = idx # idx-1 + 1 for the sake of indexing later
            if idx-2 >= 0: #to avoid runtime errors
              if (taggedWords[idx-2][1][:2] == "DT") | (taggedWords[idx-2][1][:2] == "JJ") | (taggedWords[idx-2][1][:2] == "NN"): #if there's a determiner or adjective next to the noun
                nounStart = idx-2
                if idx-3 >= 0: #to avoid runtime errors
                  if (taggedWords[idx-3][1][:2] == "DT") | (taggedWords[idx-3][1][:2] == "JJ"): #if there's a determiner next to the adjective
                    nounStart = idx-3
                  if idx-4 >=0:
                    if taggedWords[idx-4][1][:2] == "DT":
                      nounStart = idx-4
          taggedWords[idx], taggedWords[idx+1] = taggedWords[idx+1], taggedWords[idx] #swap pronoun and verb positions
          taggedWords[idx+2:idx+2] = taggedWords[nounStart:nounEnd] #add noun stuff after where pronoun used to be
          del taggedWords[nounStart:nounEnd] #remove ealier noun elements
          idx +=1 #so I don't recount verb
    idx +=1
  print("Pronoun+Verb+Noun:", taggedWords)

  #Rule 4: Adjectives After Nouns
  idx = 0
  while idx < len(taggedWords) - 1: #for every tuple (word) in Array
    if taggedWords[idx][1][:2] == "JJ": #if current word is an adjective (can tell general tag by first 2 letters of acronym)
      if taggedWords[idx+1][1][:2] == "NN": #if the proceding word is a noun #how do I avoid an error if adjective is last word in list?
        taggedWords[idx], taggedWords[idx+1] = taggedWords[idx+1], taggedWords[idx]
    idx += 1
  print("Adjectives after Nouns:", taggedWords)

  #Rule 5: "The" Before Every Non-Proper Noun
  idx = 0
  while idx < len(taggedWords):
    if (taggedWords[idx][1][:2] == "NN") & (taggedWords[idx][1][:3] != "NNP"): #if word is a noun and not a proper noun
      if idx == 0:
        taggedWords[idx:idx] = nltk.pos_tag(nltk.word_tokenize("the"))
        idx += 1 #so we don't recount nouns
      else: #to avoid runtime errors
        if (taggedWords[idx - 1][1] != "DT") & (taggedWords[idx - 1][1][:2] != "NN"): #if noun doesn't already have an "a" or "the" in front of it and if it doesn't have another noun in front of it, waiting to be turned into noun+of+noun
          taggedWords[idx:idx] = nltk.pos_tag(nltk.word_tokenize("the"))
          idx += 1 #so we don't recount nouns
    idx += 1
  print("Add 'the'", taggedWords)


  #Rule 6: Attributive Nouns
  idx = 0
  while idx < len(taggedWords) - 1:
    if taggedWords[idx][1][:2] == "NN": #if current word is a noun
      if taggedWords[idx+1][1][:2] == "NN": #if directly adjacent word is a noun
        taggedWords[idx], taggedWords[idx+1] = taggedWords[idx+1], taggedWords[idx] #swap the nouns
        #print(taggedWords)
        taggedWords[idx+1:idx+1] = nltk.pos_tag(nltk.word_tokenize("of")) #insert "of" between the two nouns
        #print(taggedWords)
    idx += 1
  print("Attributive Nouns:", taggedWords)


In [265]:
print(nltk.pos_tag(nltk.word_tokenize("a cat the cat")))

[('a', 'DT'), ('cat', 'NN'), ('the', 'DT'), ('cat', 'NN')]


In [203]:
englishNegAndAff = [("no one", "someone"), ("nothing", "something"), ("none", "some"), ("neither", "also"), ("never", "always"), ("not", "")] #no = exception. If no one --> someone. if no + noun --> none

In [202]:
spanishNegAndAff = [("nadie", "algien"), ("nada", "algo"), ("ningún", "algún"), ("ninguna", "alguna"), ("ningunas", "algunas"), ("ningunos", "algunos"), ("tampoco", "también"), ("nunca", "siempre")] #exception = no

[nltk Acronym Meanings](https://www.guru99.com/pos-tagging-chunking-nltk.html#:~:text=POS%20Tagging%20(Parts%20of%20Speech,is%20also%20called%20grammatical%20tagging.)

[Grammar Diff Btwn Spanish and English](https://www.thoughtco.com/grammatical-differences-between-spanish-and-english-4119326#:~:text=Word%20order%20is%20less%20fixed,subjunctive%20mood%20than%20English%20does.)

***Sometimes Pronouns Come Before Verbs and Nouns After***

Lo escribió Cervantes.
*   Megan wrote it (the book) --> It (the book) wrote Cervantes
*   NNP VBD PRP --> PRP VBD NNP

"No recuerdo el momento en que salió Pablo"
*   I don't remember the moment in which Pablo left --> I don't remember the moment in which left Pablo
*   (PRP VBP RB VB DT NN IN WDT) NNP VBD --> (same) VBD NNP

***Spanish Uses Double Negatives***

* Apenas come. (She barely eats.)
* Apenas come nada. (She barely eats anything --> she barely eats nothing)
* No tengo ninguno. (I don't have any --> I don't have none)
* Nadie sabe eso. (Nobody knows that.)
* Jamás fumo. (I never smoke.)
* Tampoco comió. (She didn't eat either.)
* Tampoco comió nada. (She didn't eat anything either --> She didn't eat nothing either)
* No habló. (He didn't speak.)
* No dijo nada. (He said nothing --> He didn't say nothing)
* No le dijo nada a nadie. (He didn't say anything to anybody --> He didn't say nothing to nobody)
* No compro ninguno. (I'm not buying any --> I'm not buying none)
* Nunca le compra nada a nadie. (She never buys anything for anybody --> She never buy nothing for nobody)
* No come ni siquiera pan. (He doesn't even eat bread --> He doesnt --honestly I don't even know how to translate this literally)
* Ni siquiera come pan. (He doesn't even eat bread.)

***Can Omit Pronouns if Verb is Conjugated***

* No compro ninguno. (I'm not buying any --> I'm not buying none)
* Nunca le compra nada a nadie. (She never buys anything for anybody --> She never buy nothing for nobody)
* No come ni siquiera pan. (He doesn't even eat bread --> He doesnt --honestly I don't even know how to translate this literally)
* Ni siquiera come pan. (He doesn't even eat bread.)

*Especially* Don't need "I"

***Attributive Nouns***
Not coffee cup --> taza para cafe --> cup of coffee

if two nouns next to each other --> switch nouns and add "of" (para/de) between

* In some cases, this is accomplished by Spanish having adjectival forms that don't exist in English. For example, informático can be the equivalent of "computer" as an adjective, so a computer table is a mesa informática.

***Tacking Pronouns to End of Verbs When "To"***
 * darmelo = give it to me --> give me it
 * escribamostelo = let's write it to you --> let's write you it

***bold text***

In [ ]:
def toEnglishStyle(taggedWords):
  print("under development")

### **English Data**

In [ ]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

**Syntax** **trees**

In [ ]:
import os
import matplotlib as mpl
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
### CREATE VIRTUAL DISPLAY ###
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8
Err:1 http://security.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8
  404  Not Found [IP: 91.189.88.142 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/x/xorg-server/xvfb_1.19.6-1ubuntu4.8_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [ ]:
# EXTRA STUFF TO DISPLAY NLTK SYNTAX TREES #  
%matplotlib inline
### INSTALL GHOSTSCRIPT ) ###
!apt install ghostscript python3-tk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.26~dfsg+0-0ubuntu0.18.04.14).
python3-tk is already the newest version (3.6.9-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [ ]:
chunked_sentence = '(S (NP this tree) (VP (V is) (AdjP pretty)))'

from nltk.tree import Tree
from IPython.display import display
tree = Tree.fromstring(str(chunked_sentence))
display(tree)

TclError: ignored

Tree('S', [Tree('NP', ['this', 'tree']), Tree('VP', [Tree('V', ['is']), Tree('AdjP', ['pretty'])])])

In [ ]:
Sentence = "The white dog ran"
tokenized = nltk.word_tokenize(Sentence)
print(tokenized)
tagged = nltk.pos_tag(tokenized)
print(tagged)
chunkGram = r"""Chunk: {<DT\w?>*<JJ\w?>}""" #creating a chunk
chunkParser = nltk.RegexpParser(chunkGram) #parsing the chunk
#chunked = chunkParser.parse(tagged)
for tree in chunkParser.parse(tagged): #creates a syntax tree for the parsed chunk
    print(tree)
    #tree.draw()



['The', 'white', 'dog', 'ran']
[('The', 'DT'), ('white', 'JJ'), ('dog', 'NN'), ('ran', 'VBD')]
(Chunk The/DT white/JJ)
('dog', 'NN')
('ran', 'VBD')


In [ ]:
Sentence = "white the ran dog."
tokenized = nltk.word_tokenize(Sentence)
print(tokenized)
tagged = nltk.pos_tag(tokenized)
print(tagged)

['white', 'the', 'ran', 'dog', '.']
[('white', 'JJ'), ('the', 'DT'), ('ran', 'NN'), ('dog', 'NN'), ('.', '.')]


### **Spanish Data?**

In [ ]:
!git clone https://github.com/alvations/spaghetti-tagger.git

fatal: destination path 'spaghetti-tagger' already exists and is not an empty directory.


In [ ]:
from google.colab import drive
drive.mount('/cont
ent/drive')

Mounted at /content/drive


In [ ]:
nltk.corpus.cess_esp.words()

['El', 'grupo', 'estatal', 'Electricité_de_France', ...]

In [ ]:
nltk.corpus.cess_esp.sents()

[['El', 'grupo', 'estatal', 'Electricité_de_France', '-Fpa-', 'EDF', '-Fpt-', 'anunció', 'hoy', ',', 'jueves', ',', 'la', 'compra', 'del', '51_por_ciento', 'de', 'la', 'empresa', 'mexicana', 'Electricidad_Águila_de_Altamira', '-Fpa-', 'EAA', '-Fpt-', ',', 'creada', 'por', 'el', 'japonés', 'Mitsubishi_Corporation', 'para', 'poner_en_marcha', 'una', 'central', 'de', 'gas', 'de', '495', 'megavatios', '.'], ['Una', 'portavoz', 'de', 'EDF', 'explicó', 'a', 'EFE', 'que', 'el', 'proyecto', 'para', 'la', 'construcción', 'de', 'Altamira_2', ',', 'al', 'norte', 'de', 'Tampico', ',', 'prevé', 'la', 'utilización', 'de', 'gas', 'natural', 'como', 'combustible', 'principal', 'en', 'una', 'central', 'de', 'ciclo', 'combinado', 'que', 'debe', 'empezar', 'a', 'funcionar', 'en', 'mayo_del_2002', '.'], ...]

In [ ]:
nltk.download("cess_esp")

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


True

In [ ]:
from nltk.corpus import cess_esp as cess
from nltk import UnigramTagger as ut
from nltk import BigramTagger as bt

In [ ]:
# Read the corpus into a list, 
# each entry in the list is one sentence.
cess_sents = cess.tagged_sents()

In [ ]:
# Train the unigram tagger
uni_tag = ut(cess_sents)

In [ ]:
spanishSentence = "el perro blanco corrió"

In [ ]:
#tokenized = spanishSentence.split()
tokenized = nltk.word_tokenize(spanishSentence)
print(tokenized)
tagged1 = sp.pos_tag(tokenized)
print(tagged1)
tagged2 = mytagger_uni.tag(tokenized)
print(tagged2)
tagged4 = mytagger_bi.tag(tokenized)
print(tagged4)
print(tagged4[0])
print(tagged4[0][0]) #How to parce through intervals of the tuple in the array

['el', 'perro', 'blanco', 'corrió']
[('el', 'da0ms0'), ('perro', 'ncms000'), ('blanco', 'aq0ms0'), ('corrió', 'vmis3s0')]
[('el', 'da0ms0'), ('perro', 'ncms000'), ('blanco', 'aq0ms0'), ('corrió', 'vmis3s0')]
[('el', 'da0ms0'), ('perro', 'ncms000'), ('blanco', 'aq0ms0'), ('corrió', 'vmis3s0')]
('el', 'da0ms0')
el


In [ ]:
femSentence = "la niña es fea"
tokenized1 = femSentence.split()
print(tokenized1)
tagged3 = sp.pos_tag(tokenized1)
print(tagged3)

['la', 'niña', 'es', 'fea']
[('la', 'da0fs0'), ('niña', 'ncfs000'), ('es', 'vsip3s0'), ('fea', 'aq0fs0')]


In [ ]:
# Split corpus into training and testing set.
train = int(len(cess_sents)*90/100) # 90%

# Train a bigram tagger with only training data.
bi_tag = bt(cess_sents[:train], backoff=uni_tag)

# Evaluates on testing data remaining 10%
bi_tag.evaluate(cess_sents[train+1:])

# Using the tagger.
bi_tag.tag(Sentence.split(" "))

[('el', 'da0ms0'),
 ('perro', 'ncms000'),
 ('blanco', 'aq0ms0'),
 ('corrió', 'vmis3s0')]

In [ ]:
nltk.corpus.genesis.tagged_words()

AttributeError: ignored

# **Sanskrit Data**

Mounting Drive and importing text files of Sanskrit texts

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gItA = open("/content/drive/MyDrive/Colab Notebooks/ Final Project/Colab Notebooks/bhagavadgItA.txt","r")
meghadhUta = open("/content/drive/MyDrive/Colab Notebooks/ Final Project/Colab Notebooks/meghadUta.txt","r")
rAmAyaNa = open("/content/drive/MyDrive/Colab Notebooks/ Final Project/Colab Notebooks/rAmAyaNa.txt", "r")

SyntaxError: ignored

comment to delete later: trying to figure out how to use repository here : https://github.com/ad2476/pos-research which has a sanskrit tagger. 